# Проект "Анализ сервиса вопросов и ответов по программированию"

Проект автоматически проверяется в тренажёре SQL. Работа идёт с базой данных, которая хранит информацию сервиса вопросов и ответов о программировании. Данный сервис похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.

В проекте 20 заданий на составление запросов к БД и проведение аналитической работы.

### Цель проекта

С помощью SQL рассчитать и визуализировать ключевые метрики сервис-системы вопросов и ответов о программировании

### ER-диаграмма
<img src='Image2.png' width=800, heigth=700>


### Описание таблиц

`stackoverflow.badges` - Хранит информацию о значках, которые присуждаются за разные достижения

`stackoverflow.post_types` - Содержит информацию о типе постов: Question или Answer

`stackoverflow.posts` - Содержит информацию о постах

`stackoverflow.users` - Содержит информацию о пользователях

`stackoverflow.votes` - Содержит информацию о голосах за посты

`stackoverflow.vote_types` - Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту

### Выполнение заданий

#### 1. Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в `Закладки`
```sql
SELECT COUNT(p.id)
FROM stackoverflow.posts AS p 
JOIN stackoverflow.post_types AS t ON p.post_type_id=t.id
WHERE t.type='Question'
AND (p.score > 300 OR p.favorites_count >=100);
```

#### 2. Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа
```sql
SELECT ROUND(AVG(count_question))
FROM (
    SELECT creation_date::date AS day,
           COUNT(p.id) AS count_question
    FROM stackoverflow.posts AS p 
    JOIN stackoverflow.post_types AS t ON p.post_type_id=t.id
    WHERE t.type='Question'
    AND creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
    GROUP BY day) AS november;
```

#### 3. Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей
```sql
SELECT COUNT (DISTINCT u.id)
FROM stackoverflow.users AS u
LEFT JOIN stackoverflow.badges AS b ON u.id=b.user_id
WHERE b.creation_date::date = u.creation_date::date;
```

#### 4. Сколько уникальных постов пользователя с именем `Joel Coehoorn` получили хотя бы один голос?
```sql
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON u.id=p.user_id
JOIN stackoverflow.votes AS v ON p.id=v.post_id
WHERE display_name = 'Joel Coehoorn';
```

#### 5. Выгрузите все поля таблицы `vote_types`. Добавьте к таблице поле `rank`, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю` id`
```sql
SELECT *,
       RANK() OVER (ORDER BY id DESC) AS rnk
FROM stackoverflow.vote_types
ORDER BY id;
```

#### 6. Отберите 10 пользователей, которые поставили больше всего голосов типа `Close`. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя
```sql
SELECT u.id AS username,
       COUNT(v.id) AS vote_cnt
FROM stackoverflow.votes AS v
LEFT JOIN stackoverflow.vote_types AS vt ON vt.id=v.vote_type_id
LEFT JOIN stackoverflow.users AS u ON v.user_id=u.id
WHERE vt.name='Close'
GROUP BY username
ORDER BY vote_cnt DESC,
         username DESC
LIMIT 10;
```

#### 7. Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
* идентификатор пользователя;
* число значков;
* место в рейтинге — чем больше значков, тем выше рейтинг.

**Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя**
```sql
WITH badges AS (
    SELECT DISTINCT user_id AS username,
       COUNT(id) AS badges_cnt
    FROM stackoverflow.badges AS b
    WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
    GROUP BY username
    ORDER BY badges_cnt DESC, username
    )
SELECT username,
       badges_cnt,
       DENSE_RANK() OVER(ORDER BY badges_cnt DESC)
FROM badges
LIMIT 10;
```

#### 8. Сколько в среднем очков получает пост каждого пользователя? Сформируйте таблицу из следующих полей:
* заголовок поста;
* идентификатор пользователя;
* число очков поста;
* среднее число очков пользователя за пост, округлённое до целого числа.

**Не учитывайте посты без заголовка, а также те, что набрали ноль очков**
```sql
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id)) AS score_avg
FROM  stackoverflow.posts
WHERE score!=0 AND title IS NOT NULL;
```

#### 9. Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список
```sql
WITH 
users AS (SELECT DISTINCT u.id AS username,
                 COUNT(b.id) AS badges_cnt
         FROM stackoverflow.users AS u
         JOIN stackoverflow.badges AS b ON u.id=b.user_id
         GROUP BY username
         HAVING  COUNT(b.id)>1000)
SELECT p.title AS post_name
FROM stackoverflow.posts AS p
JOIN users ON username=p.user_id
WHERE p.title IS NOT NULL;
```

#### 10. Напишите запрос, который выгрузит данные о пользователях из Канады. Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
* пользователям с числом просмотров меньше 100 — группу 3.

**Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу**
```sql
SELECT id,
       views,
       CASE
            WHEN views>=350 THEN 1
            WHEN views>=100 THEN 2
            ELSE 3
       END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0;
```

#### 11. Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора
```sql
WITH 
users AS(SELECT id,
                views,
                CASE
                    WHEN views>=350 THEN 1
                    WHEN views>=100 THEN 2
                    ELSE 3
                END AS group_name
         FROM stackoverflow.users
         WHERE location LIKE '%Canada%' AND views != 0),
leaders AS (SELECT id,
                   views,
                   group_name,
                   MAX(views) OVER(PARTITION BY group_name) AS max_views
            FROM users)
SELECT id,
       group_name,
       views
FROM leaders
WHERE views=max_views
ORDER BY views DESC,id;
```

#### 12. Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
* номер дня;
* число пользователей, зарегистрированных в этот день;
* сумму пользователей с накоплением
```sql
WITH 
row_data AS (SELECT *,
                    EXTRACT(DAY FROM creation_date::date) AS registration
             FROM stackoverflow.users
             WHERE DATE_TRUNC('month', creation_date)::date ='2008-11-01')
    
SELECT registration,
       COUNT (id) AS id_cnt,
       SUM (COUNT (id)) OVER(ORDER BY registration) AS id_total
FROM row_data
GROUP BY registration
ORDER BY registration;
```

#### 13. Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста
```sql
WITH 
users AS (SELECT DISTINCT u.id AS username,
                 MIN(p.creation_date) OVER(PARTITION BY p.user_id) AS first_post,
                 u.creation_date AS day_registr
          FROM stackoverflow.posts AS p
          JOIN stackoverflow.users AS u ON u.id=p.user_id)
SELECT username,
       first_post - day_registr
FROM users;
```

#### 14. Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров
```sql
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count) AS views_cnt
FROM stackoverflow.posts
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY views_cnt DESC;
```

#### 15. Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений `user_id`. Отсортируйте результат по полю с именами в лексикографическом порядке
```sql
WITH 
row_data AS (SELECT u.display_name AS username,
                    p.user_id AS idname, 
                    p.id AS postname,
                    u.creation_date::date + INTERVAL'1 month' AS one_month_regs,
                    p.creation_date::date AS post_date
             FROM stackoverflow.posts p
             LEFT JOIN stackoverflow.users u ON p.user_id=u.id
             LEFT JOIN stackoverflow.post_types pt ON pt.id=p.post_type_id
             WHERE pt.type='Answer')
 SELECT username,
        COUNT(DISTINCT idname)
 FROM row_data
 WHERE post_date<=one_month_regs
 GROUP BY username
 HAVING COUNT(postname) >100;
```

#### 16. Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию
```sql
SELECT DATE_TRUNC('month', p1.creation_date)::date AS month_post,
       COUNT(p1.id)
FROM stackoverflow.posts p1
WHERE p1.user_id IN (
    SELECT DISTINCT u.id AS username
    FROM stackoverflow.users u
    JOIN stackoverflow.posts p ON u.id = p.user_id
    WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
    AND  DATE_TRUNC('month', p.creation_date)::date = '2008-12-01')
GROUP BY 1
ORDER BY 1 DESC;
```

#### 17. Используя данные о постах, выведите несколько полей:
* идентификатор пользователя, который написал пост;
* дата создания поста;
* количество просмотров у текущего поста;
* сумма просмотров постов автора с накоплением.

**Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста**
```sql
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts p;
```

#### 18. Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост
```sql
WITH 
posts AS(SELECT user_id,
                COUNT(DISTINCT DATE_TRUNC('day', creation_date)::date) AS day_cnt
         FROM stackoverflow.posts
         WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-12-01' AND '2008-12-07'
         GROUP BY user_id)
SELECT ROUND(AVG(day_cnt))
FROM posts;
```

#### 19. На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
* Номер месяца.
* Количество постов за месяц.
* Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим. Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой
```sql
WITH 
posts AS (SELECT EXTRACT(MONTH FROM creation_date) AS month_post,
                 COUNT(id) AS post_cnt
          FROM stackoverflow.posts
          WHERE DATE_TRUNC('month', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31'
          GROUP BY month_post)
SELECT month_post,
       post_cnt,
       ROUND(((post_cnt::numeric / LAG(post_cnt) OVER (ORDER BY month_post)) - 1) * 100,2) AS post_cnt_delta
FROM posts;
```

#### 20. Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
* номер недели;
* дата и время последнего поста, опубликованного на этой неделе
```sql
WITH 
top_user AS (SELECT user_id AS top_id,
                    COUNT(id) AS post_cnt
             FROM stackoverflow.posts
             GROUP BY user_id
             ORDER BY post_cnt DESC
             LIMIT 1),
             
top_user_posts AS (SELECT id,
                          EXTRACT(WEEK FROM creation_date) AS week_number,
                          creation_date AS date_time
                   FROM stackoverflow.posts p
                   JOIN top_user ON top_user.top_id=p.user_id
                   WHERE DATE_TRUNC('month', creation_date)::date ='2008-10-01')
                   
SELECT DISTINCT week_number,
       MAX(date_time) OVER(PARTITION BY week_number)
FROM top_user_posts;
```